# Install dependencies (using %pip magic ...)

In [ ]:
%pip install openai python-dotenv google-search-results

# Create the `%ai` Jupyter magic

In [1]:
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

def create_new_cell(contents):
    from IPython.core.getipython import get_ipython
    shell = get_ipython()
    shell.set_next_input(contents, replace=False)
            
# The class MUST call this class decorator at creation time
@magics_class
class JupyterCopilotMagics(Magics):        
    @line_cell_magic
    def ai(self, line, cell=None):               
        #print("Full access to the main IPython object:", self.shell)
        #print("Variables in the user namespace:", list(self.shell.user_ns.keys()))
        
        prompt = cell or line
        response = openai.Completion.create(
          model="text-davinci-002",
          prompt=prompt,
          temperature=0,
          max_tokens=100,
          top_p=1,
          frequency_penalty=0.5,
          presence_penalty=0
        )
        text = response["choices"][0]["text"]
        while text[0] == "\n":
            text = text[1:]
        create_new_cell(text)

get_ipython().register_magics(JupyterCopilotMagics)

In [2]:
%ai Create a python function that hashes a string using SHA256.

In [ ]:
def hash_string_sha256(string):

    # Convert the string to bytes
    string_bytes = string.encode('utf-8')

    # Use the sha256 function to hash the string
    return hashlib.sha256(string_bytes).hexdigest()

In [6]:
%ai Create a python function that writes an object into a json file pretty printed 

In [ ]:
def write_json(filename, obj):
    with open(filename, 'w') as f:
        json.dump(obj, f, indent=4)

In [3]:
from serpapi import GoogleSearch
from pathlib import Path
import json

SERP_API_KEY = os.getenv("SERP_API_KEY")

base_path = Path(".webcache")

if not base_path.exists():
    base_path.mkdir()

def search(query):
    query = query.lower().strip()
    h = hash_string_sha256(query) + ".json"
    cache_file = base_path / h
    if not cache_file.exists():
        params = {
          "engine": "google",
          "q": query,
          "api_key": SERP_API_KEY
        }

        search = GoogleSearch(params)
        results = search.get_dict()
        organic_results = results["organic_results"]
        write_json(cache_file, organic_results)
    else:
        organic_results = json.load(cache_file.open("r"))
        
        
    return organic_results